In [2]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

### PD Set Options

In [68]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [92]:
dsf1 = pd.read_csv('weeklycondosalesfix - Sheet1.csv', header=0)
dsf2 = tabula.read_pdf('Topcondoimage 10-03-2022.pdf', pages='all',lattice=True)
dsf3 = tabula.read_pdf('Topcondoimage 09-19-2022.pdf', pages='all',lattice=True)
dsf4 = tabula.read_pdf('Topcondoimage 09-12-2022 alternate.pdf', pages='all',lattice=True)
dsf5 = tabula.read_pdf('Topcondoimage 09-06-2022.pdf', pages='all',lattice=True)

In [93]:
df1 = dsf1
df2 = dsf2[0]
df3 = dsf3[0]
df4 = dsf4[0]
df5 = dsf5[0]

In [94]:
#df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df3.columns = df3.iloc[0]
df4.columns = df4.iloc[0]
df5.columns = df5.iloc[0]

In [95]:
#df1 = df1.drop(df1.index[0])
df2 = df2.drop(df2.index[0])
df3 = df3.drop(df3.index[0])
df4 = df4.drop(df4.index[0])
df5 = df5.drop(df5.index[0])

In [96]:
#df1 = df1.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[: , 1:]
df4 = df4.iloc[: , 1:]
df5 = df5.iloc[: , 1:]

In [99]:
df = pd.concat([df2, df3, df4, df5], ignore_index=True)

In [100]:
# df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

In [101]:
df = pd.concat([df, df1])

In [102]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address
0,One Thousand Museum\r1000 Biscayne Blvd PH-57\rMiami,5/27/2022,9/29/2022,116,"$19,500,000.00",9200,"$2,119.57",Julian Johnston,The Corcoran Group,Christopher Wands,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd PH-57 Miami
1,SLS Brickell\r1300 S Miami Ave UPH3\rMiami,6/28/2022,9/29/2022,63,"$4,600,000.00",2268,"$2,028.22",MariaClaudia Fernandez,One Sotheby's Int'l Realty,David Freed,Keller Williams Miami Beach Realty,SLS Brickell 1300 S Miami Ave UPH3 Miami
2,Jade Signature\r16901 Collins Ave 3705\rSunny Isles Beach,12/8/2021,9/28/2022,226,"$4,500,000.00",3405,"$1,321.59",Marina Osmacova,MIAMI VIP REALTY LLC,Steven Seigel,"Compass Florida, LLC",Jade Signature 16901 Collins Ave 3705 Sunny Isles Beach
3,Green Diamond\r4775 Collins Ave 4003\rMiami Beach,4/26/2022,9/30/2022,58,"$3,550,000.00",1980,"$1,792.93",Claudia Whitechurch,Coldwell Banker Realty,Menachem Fellig,"Compass Florida, LLC",Green Diamond 4775 Collins Ave 4003 Miami Beach
4,Prive\r5000 Island Estates Dr 1007\rAventura,6/30/2022,9/30/2022,29,"$3,500,000.00",2830,"$1,236.75",Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Prive 5000 Island Estates Dr 1007 Aventura
5,The Setai\r101 20th St 2201\rMiami Beach,6/26/2022,9/27/2022,31,"$3,300,000.00",1198,"$2,754.59",Cecilia Solorzano,Suma Luxury Realty,Matt Lill,The Corcoran Group,The Setai 101 20th St 2201 Miami Beach
6,Fisher Island\r19123 Fisher Island Dr 19123\rMiami Beach,4/19/2022,9/27/2022,118,"$2,430,000.00",1875,"$1,296.00",Nataly Stuart,"Socialite Realty, LLC",Gregory Gringruz,The Island,Fisher Island 19123 Fisher Island Dr 19123 Miami Beach
7,Murano Grande\r400 Alton Rd TH-4A\rMiami Beach,4/12/2022,9/30/2022,127,"$1,920,000.00",1912,"$1,004.18",Barbara Smith,"Golden Realty Group, Inc.",Oliver Davis,Keller Williams Miami Beach Realty,Murano Grande 400 Alton Rd TH-4A Miami Beach
8,St Tropez on the Bay\r150 Sunny Isle Blvd 1-UPH3\rSunny Isles Beach,4/5/2022,9/27/2022,76,"$1,700,000.00",3020,$562.91,Bento Queiroz,"Compass Florida, LLC",Gina Nash,Aston Rose Sports And Entertai,St Tropez on the Bay 150 Sunny Isle Blvd 1-UPH3 Sunny Isles Beach
9,R0each Condo\r68 SE 6th St 2603\rMiami,11/11/2021,9/27/2022,54,"$1,700,000.00",1891,$899.00,Claudia Lopez,Douglas Elliman,Ross Milroy,The Corcoran Group,R0each Condo 68 SE 6th St 2603 Miami


Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [103]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [104]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
0,One Thousand Museum\r1000 Biscayne Blvd PH-57\rMiami,5/27/2022,9/29/2022,116,"$19,500,000.00",9200,"$2,119.57",Julian Johnston,The Corcoran Group,Christopher Wands,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd PH-57 Miami,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami
0,The Gables Club\n60 Edgewater Dr TSE\nCoral Gables,11/17/2021,9/21/2022,265,"$7,270,000",7980,$911,Allison Blumenthal,x,Ashley Cusack,x,NaN,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami
1,SLS Brickell\r1300 S Miami Ave UPH3\rMiami,6/28/2022,9/29/2022,63,"$4,600,000.00",2268,"$2,028.22",MariaClaudia Fernandez,One Sotheby's Int'l Realty,David Freed,Keller Williams Miami Beach Realty,SLS Brickell 1300 S Miami Ave UPH3 Miami,SLS Brickell,1300 S Miami Ave UPH3 Miami
1,Continuum on South Beach\n100 S Pointe Drive 3905\nMiami Beach,5/23/2022,9/19/2022,67,"$6,500,000",1793,3625,Roland Ortiz,x,Sonia Toth,x,NaN,SLS Brickell,1300 S Miami Ave UPH3 Miami
2,Jade Signature\r16901 Collins Ave 3705\rSunny Isles Beach,12/8/2021,9/28/2022,226,"$4,500,000.00",3405,"$1,321.59",Marina Osmacova,MIAMI VIP REALTY LLC,Steven Seigel,"Compass Florida, LLC",Jade Signature 16901 Collins Ave 3705 Sunny Isles Beach,Jade Signature,16901 Collins Ave 3705 Sunny Isles Beach
2,W South Beach\n2201 Collins Ave 726-728-730\nMiami Beach,6/20/2022,9/23/2022,58,"$6,084,000",3082,2394,Angelica Garcia,x,Christopher Adeleke,x,NaN,Jade Signature,16901 Collins Ave 3705 Sunny Isles Beach
3,Green Diamond\r4775 Collins Ave 4003\rMiami Beach,4/26/2022,9/30/2022,58,"$3,550,000.00",1980,"$1,792.93",Claudia Whitechurch,Coldwell Banker Realty,Menachem Fellig,"Compass Florida, LLC",Green Diamond 4775 Collins Ave 4003 Miami Beach,Green Diamond,4775 Collins Ave 4003 Miami Beach
3,Bal Harbour Tower\n9999 Collins Avenue PH 2H\nBal Harbour,4/19/2022,9/22/2022,106,"$6,000,000",4980,1205,Agnieska Gray,x,Stacy Robins,x,NaN,Green Diamond,4775 Collins Ave 4003 Miami Beach
4,Prive\r5000 Island Estates Dr 1007\rAventura,6/30/2022,9/30/2022,29,"$3,500,000.00",2830,"$1,236.75",Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Prive 5000 Island Estates Dr 1007 Aventura,Prive,5000 Island Estates Dr 1007 Aventura
4,Porsche Design Tower\n18555 Collins Avenue 1701\nSunny Isles Beach,5/15/2020,9/19/2022,846,"$5,750,000",4154,1384,Heloisa Arazi,x,Allison Turk PA,x,NaN,Prive,5000 Island Estates Dr 1007 Aventura


In [105]:
x = df['Close Date'].str.startswith('9')
df = df[x]

In [106]:
len(df)

44

In [107]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [108]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [109]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/22809927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)


In [110]:
df['loc']

0                             (1000 Biscayne Blvd #57, Miami, FL 33132, USA, (25.784171, -80.1900783))
0                             (1000 Biscayne Blvd #57, Miami, FL 33132, USA, (25.784171, -80.1900783))
1                           (1300 S Miami Ave, Miami, FL 33130, USA, (25.7612047, -80.19359659999999))
1                           (1300 S Miami Ave, Miami, FL 33130, USA, (25.7612047, -80.19359659999999))
2               (16901 Collins Ave #3705, Sunny Isles Beach, FL 33160, USA, (25.9327557, -80.1210054))
2               (16901 Collins Ave #3705, Sunny Isles Beach, FL 33160, USA, (25.9327557, -80.1210054))
3                   (4775 Collins Ave APT 4003, Miami Beach, FL 33140, USA, (25.8227293, -80.1226745))
3                   (4775 Collins Ave APT 4003, Miami Beach, FL 33140, USA, (25.8227293, -80.1226745))
4                   (5000 Island Estates Dr #1007, Aventura, FL 33160, USA, (25.9470842, -80.1306458))
4                   (5000 Island Estates Dr #1007, Aventura, FL 33160, US

In [111]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/1957931448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)


Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find  coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [112]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point'],
      dtype='object')

In [113]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)



/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/3326340165.py:1: SettingWit

In [40]:
df.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 41, 43, 44, 47],
           dtype='int64')

In [114]:
### Not sure why this works the way it does. But if I want to simply reset the index because the numbers are askew,
### I need to set both 'drop' and 'inplace' to 'True'
df.reset_index(drop=True, inplace=True)

In [116]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
0,One Thousand Museum\r1000 Biscayne Blvd PH-57\rMiami,5/27/2022,9/29/2022,116,"$19,500,000.00",9200,"$2,119.57",Julian Johnston,The Corcoran Group,Christopher Wands,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd PH-57 Miami,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami,"(1000 Biscayne Blvd #57, Miami, FL 33132, USA, (25.784171, -80.1900783))","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
1,The Gables Club\n60 Edgewater Dr TSE\nCoral Gables,11/17/2021,9/21/2022,265,"$7,270,000",7980,$911,Allison Blumenthal,x,Ashley Cusack,x,NaN,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami,"(1000 Biscayne Blvd #57, Miami, FL 33132, USA, (25.784171, -80.1900783))","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
2,SLS Brickell\r1300 S Miami Ave UPH3\rMiami,6/28/2022,9/29/2022,63,"$4,600,000.00",2268,"$2,028.22",MariaClaudia Fernandez,One Sotheby's Int'l Realty,David Freed,Keller Williams Miami Beach Realty,SLS Brickell 1300 S Miami Ave UPH3 Miami,SLS Brickell,1300 S Miami Ave UPH3 Miami,"(1300 S Miami Ave, Miami, FL 33130, USA, (25.7612047, -80.19359659999999))","(25.7612047, -80.19359659999999, 0.0)",25.761205,-80.193597,0.0
3,Continuum on South Beach\n100 S Pointe Drive 3905\nMiami Beach,5/23/2022,9/19/2022,67,"$6,500,000",1793,3625,Roland Ortiz,x,Sonia Toth,x,NaN,SLS Brickell,1300 S Miami Ave UPH3 Miami,"(1300 S Miami Ave, Miami, FL 33130, USA, (25.7612047, -80.19359659999999))","(25.7612047, -80.19359659999999, 0.0)",25.761205,-80.193597,0.0
4,Jade Signature\r16901 Collins Ave 3705\rSunny Isles Beach,12/8/2021,9/28/2022,226,"$4,500,000.00",3405,"$1,321.59",Marina Osmacova,MIAMI VIP REALTY LLC,Steven Seigel,"Compass Florida, LLC",Jade Signature 16901 Collins Ave 3705 Sunny Isles Beach,Jade Signature,16901 Collins Ave 3705 Sunny Isles Beach,"(16901 Collins Ave #3705, Sunny Isles Beach, FL 33160, USA, (25.9327557, -80.1210054))","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0
5,W South Beach\n2201 Collins Ave 726-728-730\nMiami Beach,6/20/2022,9/23/2022,58,"$6,084,000",3082,2394,Angelica Garcia,x,Christopher Adeleke,x,NaN,Jade Signature,16901 Collins Ave 3705 Sunny Isles Beach,"(16901 Collins Ave #3705, Sunny Isles Beach, FL 33160, USA, (25.9327557, -80.1210054))","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0
6,Green Diamond\r4775 Collins Ave 4003\rMiami Beach,4/26/2022,9/30/2022,58,"$3,550,000.00",1980,"$1,792.93",Claudia Whitechurch,Coldwell Banker Realty,Menachem Fellig,"Compass Florida, LLC",Green Diamond 4775 Collins Ave 4003 Miami Beach,Green Diamond,4775 Collins Ave 4003 Miami Beach,"(4775 Collins Ave APT 4003, Miami Beach, FL 33140, USA, (25.8227293, -80.1226745))","(25.8227293, -80.1226745, 0.0)",25.822729,-80.122675,0.0
7,Bal Harbour Tower\n9999 Collins Avenue PH 2H\nBal Harbour,4/19/2022,9/22/2022,106,"$6,000,000",4980,1205,Agnieska Gray,x,Stacy Robins,x,NaN,Green Diamond,4775 Collins Ave 4003 Miami Beach,"(4775 Collins Ave APT 4003, Miami Beach, FL 33140, USA, (25.8227293, -80.1226745))","(25.8227293, -80.1226745, 0.0)",25.822729,-80.122675,0.0
8,Prive\r5000 Island Estates Dr 1007\rAventura,6/30/2022,9/30/2022,29,"$3,500,000.00",2830,"$1,236.75",Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Prive 5000 Island Estates Dr 1007 Aventura,Prive,5000 Island Estates Dr 1007 Aventura,"(5000 Island Estates Dr #1007, Aventura, FL 33160, USA, (25.9470842, -80.1306458))","(25.9470842, -80.1306458, 0.0)",25.947084,-80.130646,0.0
9,Porsche Design Tower\n18555 Collins Avenue 1701\nSunny Isles Beach,5/15/2020,9/19/2022,846,"$5,750,000",4154,1384,Heloisa Arazi,x,Allison Turk PA,x,NaN,Prive,5000 Island Estates Dr 1007 Aventura,"(5000 Island Estates Dr #1007, Aventura, FL 33160, USA, (25.9470842, -80.1306458))","(25.9470842, -80.13

In [117]:
df.at[18,'building_name']=('Reach')
df.at[19,'building_name']=('Reach')
df.at[29,'building_name']=('Balmoral')
df.at[39,'building_name']=('Carbonel')

# df.at[9,'address_only']=('Vizcayne South Condo 253 NE 2nd 4408 Miami')
# df.at[19,'address_only']=('Trump Royale 18201 Collins Ave 806 Sunny Isles Beach')
# df.at[29,'address_only']=('Trump Royale 18201 Collins Ave 3809 Sunny Isles Beach')
# df.at[39,'address_only']=('South Pointe Towers 400 S Pointe Dr 906 Miami Beach')
# df.at[45,'address_only']=('The Fairchild 3581 E Glencoe St 408 Miami')

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [118]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'September 2022 Top Miami-Dade Condo Sales ')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Address: {bold_start}{info[-1]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Square Feet: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [119]:
m.save('index.html')

In [61]:
### Clean DF for graphic ###

In [120]:
df['Sale Price int'] = df['Sale Price']
df['price_per_sqft_int'] = df['price_per_sqft']

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/2911951518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price']
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/2911951518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = df['price_per_sqft']


In [121]:
df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)

df['price_per_sqft_int'] = df['price_per_sqft_int'].replace('\$','',regex=True)
df['price_per_sqft_int'] = df['price_per_sqft_int'].replace(',','',regex=True)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/310194361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/310194361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/310194361.py:4: SettingWithCopyWarning: 
A value i

In [122]:
df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/879808360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/879808360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])


In [123]:
df['days_on_market'] = pd.to_numeric(df['days_on_market'])
df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/1544171665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_on_market'] = pd.to_numeric(df['days_on_market'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_18356/1544171665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])


In [124]:
df.sort_values('Sale Price int',ascending=False)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int
0,One Thousand Museum\r1000 Biscayne Blvd PH-57\rMiami,5/27/2022,9/29/2022,116,"$19,500,000.00",9200,"$2,119.57",Julian Johnston,The Corcoran Group,Christopher Wands,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd PH-57 Miami,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami,"(1000 Biscayne Blvd #57, Miami, FL 33132, USA, (25.784171, -80.1900783))","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0,19500000.0,2119.57
20,Jade Signature\r16901 Collins Ave PH5503\rSunny Isles Beach,2/4/2022,9/16/2022,223,"$15,000,000.00",14425,"$1,633.45",Matthew Dugow,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",Jade Signature 16901 Collins Ave PH5503 Sunny Isles Beach,Jade Signature,16901 Collins Ave PH5503 Sunny Isles Beach,"(16901 Collins Ave #5503, Sunny Isles Beach, FL 33160, USA, (25.9327557, -80.1210054))","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0,15000000.0,1633.45
30,Chateau Ocean Condo\r9349 Collins Ave 304\rSurfside,9/23/2021,9/6/2022,313,"$9,200,000.00",3232,"$2,846.53",Maria Oneide Gioia,Luxury Living Realty,Alina Meledina,AGK Realty LLC,Chateau Ocean Condo 9349 Collins Ave 304 Surfside,Chateau Ocean Condo,9349 Collins Ave 304 Surfside,"(9349 Collins Ave Apt 304, Surfside, FL 33154, USA, (25.8830205, -80.1216893))","(25.8830205, -80.1216893, 0.0)",25.883021,-80.121689,0.0,9200000.0,2846.53
40,W Resort & Residence\r2201 Collins Ave PH28\rMiami Beach,11/8/2021,9/1/2022,254,"$8,750,000.00",2642,"$3,311.88",Anna Sherrill,One Sotheby's Int'l Realty,Eloy Carmenate,The Corcoran Group,W Resort & Residence 2201 Collins Ave PH28 Miami Beach,W Resort & Residence,2201 Collins Ave PH28 Miami Beach,"(2201 Collins Ave, Miami Beach, FL 33139, USA, (25.7979013, -80.1275924))","(25.7979013, -80.1275924, 0.0)",25.797901,-80.127592,0.0,8750000.0,3311.88
1,The Gables Club\n60 Edgewater Dr TSE\nCoral Gables,11/17/2021,9/21/2022,265,"$7,270,000",7980,$911,Allison Blumenthal,x,Ashley Cusack,x,NaN,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami,"(1000 Biscayne Blvd #57, Miami, FL 33132, USA, (25.784171, -80.1900783))","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0,7270000.0,911.00
3,Continuum on South Beach\n100 S Pointe Drive 3905\nMiami Beach,5/23/2022,9/19/2022,67,"$6,500,000",1793,3625,Roland Ortiz,x,Sonia Toth,x,NaN,SLS Brickell,1300 S Miami Ave UPH3 Miami,"(1300 S Miami Ave, Miami, FL 33130, USA, (25.7612047, -80.19359659999999))","(25.7612047, -80.19359659999999, 0.0)",25.761205,-80.193597,0.0,6500000.0,3625.00
5,W South Beach\n2201 Collins Ave 726-728-730\nMiami Beach,6/20/2022,9/23/2022,58,"$6,084,000",3082,2394,Angelica Garcia,x,Christopher Adeleke,x,NaN,Jade Signature,16901 Collins Ave 3705 Sunny Isles Beach,"(16901 Collins Ave #3705, Sunny Isles Beach, FL 33160, USA, (25.9327557, -80.1210054))","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0,6084000.0,2394.00
7,Bal Harbour Tower\n9999 Collins Avenue PH 2H\nBal Harbour,4/19/2022,9/22/2022,106,"$6,000,000",4980,1205,Agnieska Gray,x,Stacy Robins,x,NaN,Green Diamond,4775 Collins Ave 4003 Miami Beach,"(4775 Collins Ave APT 4003, Miami Beach, FL 33140, USA, (25.8227293, -80.1226745))","(25.8227293, -80.1226745, 0.0)",25.822729,-80.122675,0.0,6000000.0,1205.00
9,Porsche Design Tower\n18555 Collins Avenue 1701\nSunny Isles Beach,5/15/2020,9/19/2022,846,"$5,750,000",4154,1384,Heloisa Arazi,x,Allison Turk PA,x,NaN,Prive,5000 Island Estates Dr 1007 Aventura,"(5000 Island Estates Dr #1007, Aventura, FL 33160, USA, (25.9470842, -80.1306458))","(25.9470842, -80.1306458, 0.0)",25.947084,-80.130646,0.0,5750000.0,1384.00
21,Mansions at Acqualina\r17749 Collins Ave 1102\rSunny Isles Beach,7/31/2020,9/15/2022,220,"$5,500,000.00",4609,"$1,193.32",Cassio Galiza PA,Podium Realty Group INC,Marvin Anhalt,Anhalt Realty,

In [125]:
df.sort_values('days_on_market',ascending=False)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int
9,Porsche Design Tower\n18555 Collins Avenue 1701\nSunny Isles Beach,5/15/2020,9/19/2022,846,"$5,750,000",4154,1384,Heloisa Arazi,x,Allison Turk PA,x,NaN,Prive,5000 Island Estates Dr 1007 Aventura,"(5000 Island Estates Dr #1007, Aventura, FL 33160, USA, (25.9470842, -80.1306458))","(25.9470842, -80.1306458, 0.0)",25.947084,-80.130646,0.0,5750000.0,1384.00
31,Oceanside at Fisher Island\r7454 Fisher Island Dr 7454\rMiami Beach,2/21/2021,9/8/2022,541,"$5,400,000.00",2942,"$1,835.49",Evelyn Mackenzie,Aria Luxe Inc,Evelyn Mackenzie,Aria Luxe Inc,Oceanside at Fisher Island 7454 Fisher Island Dr 7454 Miami Beach,Oceanside at Fisher Island,7454 Fisher Island Dr 7454 Miami Beach,"(7454 Fisher Island Dr #7454, Miami Beach, FL 33109, USA, (25.7617, -80.13568599999999))","(25.7617, -80.13568599999999, 0.0)",25.761700,-80.135686,0.0,5400000.0,1835.49
35,Chateau Beach\r17475 Collins Ave 1002\rSunny Isles Beach,2/24/2021,9/9/2022,452,"$2,650,000.00",2164,"$1,224.58",Anna Reznik,GMT International Realty Inc,Anna Reznik,GMT International Realty Inc,Chateau Beach 17475 Collins Ave 1002 Sunny Isles Beach,Chateau Beach,17475 Collins Ave 1002 Sunny Isles Beach,"(17475 Collins Ave UNIT 1002, Sunny Isles Beach, FL 33160, USA, (25.9379684, -80.1207198))","(25.9379684, -80.1207198, 0.0)",25.937968,-80.120720,0.0,2650000.0,1224.58
30,Chateau Ocean Condo\r9349 Collins Ave 304\rSurfside,9/23/2021,9/6/2022,313,"$9,200,000.00",3232,"$2,846.53",Maria Oneide Gioia,Luxury Living Realty,Alina Meledina,AGK Realty LLC,Chateau Ocean Condo 9349 Collins Ave 304 Surfside,Chateau Ocean Condo,9349 Collins Ave 304 Surfside,"(9349 Collins Ave Apt 304, Surfside, FL 33154, USA, (25.8830205, -80.1216893))","(25.8830205, -80.1216893, 0.0)",25.883021,-80.121689,0.0,9200000.0,2846.53
29,B0almoral\r9801Collins Ave PH7\rBal Harbour,10/7/2021,9/16/2022,299,"$1,700,000.00",1682,"$1,010.70",Boris Vertsberger,"FIP Realty Services, LLC",Joelle Oiknine,One Sotheby's Int'l Realty,B0almoral 9801Collins Ave PH7 Bal Harbour,Balmoral,0almoral 9801Collins Ave PH7 Bal Harbour,"(9801 Collins Ave, Bal Harbour, FL 33154, USA, (25.8902259, -80.1228888))","(25.8902259, -80.1228888, 0.0)",25.890226,-80.122889,0.0,1700000.0,1010.70
1,The Gables Club\n60 Edgewater Dr TSE\nCoral Gables,11/17/2021,9/21/2022,265,"$7,270,000",7980,$911,Allison Blumenthal,x,Ashley Cusack,x,NaN,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami,"(1000 Biscayne Blvd #57, Miami, FL 33132, USA, (25.784171, -80.1900783))","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0,7270000.0,911.00
40,W Resort & Residence\r2201 Collins Ave PH28\rMiami Beach,11/8/2021,9/1/2022,254,"$8,750,000.00",2642,"$3,311.88",Anna Sherrill,One Sotheby's Int'l Realty,Eloy Carmenate,The Corcoran Group,W Resort & Residence 2201 Collins Ave PH28 Miami Beach,W Resort & Residence,2201 Collins Ave PH28 Miami Beach,"(2201 Collins Ave, Miami Beach, FL 33139, USA, (25.7979013, -80.1275924))","(25.7979013, -80.1275924, 0.0)",25.797901,-80.127592,0.0,8750000.0,3311.88
4,Jade Signature\r16901 Collins Ave 3705\rSunny Isles Beach,12/8/2021,9/28/2022,226,"$4,500,000.00",3405,"$1,321.59",Marina Osmacova,MIAMI VIP REALTY LLC,Steven Seigel,"Compass Florida, LLC",Jade Signature 16901 Collins Ave 3705 Sunny Isles Beach,Jade Signature,16901 Collins Ave 3705 Sunny Isles Beach,"(16901 Collins Ave #3705, Sunny Isles Beach, FL 33160, USA, (25.9327557, -80.1210054))","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0,4500000.0,1321.59
19,Lake Villa Three Condo\n739 Crandon Blvd PH1\nKey Biscayne,11/11/2021,9/19/2022,224,"$2,200,000",2220,991,Monica Slodarz,x,Roberta Fittipaldi,x,NaN,Reach,0each Condo 68 SE 6th St 2603 Miami,"(68 SE 6th St Apt 2603, Miami, FL 33131, USA, (25.7680813, -80.1926812))","(25.7680813, -80.1926812, 0.0

### Sale price sum

In [126]:
"${:,.2f}".format(df['Sale Price int'].sum())

'$185,320,500.00'

### Sale price average

In [127]:
"${:,.2f}".format(df['Sale Price int'].mean())

'$4,211,829.55'

### Price per sf average

In [128]:
"${:,.2f}".format(df['price_per_sqft_int'].mean())

'$1,449.38'

### Price per square foot min vs. max

In [129]:
"${:,.2f}".format(df['price_per_sqft_int'].max(axis=0))

'$3,625.00'

In [130]:
"${:,.2f}".format(df['price_per_sqft_int'].min(axis=0))

'$419.98'

### Sale price min and max

In [131]:
"${:,.2f}".format(df['Sale Price int'].max(axis=0))

'$19,500,000.00'

In [132]:
"${:,.2f}".format(df['Sale Price int'].min(axis=0))

'$1,650,000.00'

In [133]:
len(df)

44

In [76]:
df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/584204004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))


,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int
0,Asia 900 Brickell Key Blvd 3103 Miami,5/27/2022,8/18/2022,66.0,"$4,150,000.00",3398,"$1,221.31",Fabiola Lima,KB Realty,Ana Teresa Rodriguez,Coldwell Banker Realty,Asia 900 Brickell Key Blvd 3103 Miami,Asia,900 Brickell Key Blvd 3103 Miami,"(Asia, 900 Brickell Key Blvd APT 3103, Miami, FL 33131, USA, (25.7690295, -80.1856979))","(25.7690295, -80.1856979, 0.0)",25.769029,-80.185698,0.0,4150000.0,1221.31
1,Ocean Three Condo 18911 Collins Ave 901 Sunny Isles Beach,2/25/2022,8/17/2022,112.0,"$3,200,000.00",3945,$811.15,Madeleyne Sutton,One Sotheby's Int'l Realty,Ekaterina Brosda,"Brosda & Bentley Realtors, LLC",Ocean Three Condo 18911 Collins Ave 901 Sunny Isles Beach,Ocean Three Condo,18911 Collins Ave 901 Sunny Isles Beach,"(18911 Collins Ave APT 901, Sunny Isles Beach, FL 33160, USA, (25.9509344, -80.11988579999999))","(25.9509344, -80.11988579999999, 0.0)",25.950934,-80.119886,0.0,3200000.0,811.15
2,Roney Palace Condo 2301 Collins Ave 810 Miami Beach,6/9/2022,8/16/2022,8.0,"$2,475,000.00",1480,"$1,672.30",Ileana De La Torre,Douglas Elliman,Kezia Ceccon,Thine Agency,Roney Palace Condo 2301 Collins Ave 810 Miami Beach,Roney Palace Condo,2301 Collins Ave 810 Miami Beach,"(2301 Collins Ave APT 810, Miami Beach, FL 33139, USA, (25.7987825, -80.1271941))","(25.7987825, -80.1271941, 0.0)",25.798783,-80.127194,0.0,2475000.0,1672.30
3,Portofino Tower 300 S Pointe Dr 2606 Miami Beach,7/8/2022,8/15/2022,6.0,"$2,200,000.00",2040,"$1,078.43",Christian Brandl,London Foster Realty,Piero Ruggeri,COSMORE Florida Corp.,Portofino Tower 300 S Pointe Dr 2606 Miami Beach,Portofino Tower,300 S Pointe Dr 2606 Miami Beach,"(300 South Pointe Dr APT 2606, Miami Beach, FL 33139, USA, (25.7677332, -80.1349941))","(25.7677332, -80.1349941, 0.0)",25.767733,-80.134994,0.0,2200000.0,1078.43
4,Ocean Two Condo 19111 Collins Ave 3707 Sunny Isles Beach,3/5/2022,8/15/2022,82.0,"$1,562,500.00",2150,$801.28,Gal Kol,Compass Florida LLC,Asi Topaz,"Brokers, LLC",Ocean Two Condo 19111 Collins Ave 3707 Sunny Isles Beach,Ocean Two Condo,19111 Collins Ave 3707 Sunny Isles Beach,"(19111 Collins Ave APT 3707, Sunny Isles Beach, FL 33160, USA, (25.9534131, -80.1197571))","(25.9534131, -80.1197571, 0.0)",25.953413,-80.119757,0.0,1562500.0,801.28
5,TDR Tower II Condo 15901 Collins Ave 406 Sunny Isles Beach,5/25/2022,8/17/2022,72.0,"$1,500,000.00",2354,$637.21,Giulietta Ulloa,BHHS EWM Realty,Bernard Bell,Sunny Realty,TDR Tower II Condo 15901 Collins Ave 406 Sunny Isles Beach,TDR Tower II Condo,15901 Collins Ave 406 Sunny Isles Beach,"(15901 Collins Ave APT 406, Sunny Isles Beach, FL 33160, USA, (25.9240235, -80.1218071))","(25.9240235, -80.1218071, 0.0)",25.924024,-80.121807,0.0,1500000.0,637.21
6,Key Colony Botanica 181 Crandon Blvd 205 Key Biscayne,4/20/2022,8/16/2022,47.0,"$1,480,000.00",1600,$925.00,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",Istvan Zolcsak,One Sotheby's Int'l Realty,Key Colony Botanica 181 Crandon Blvd 205 Key Biscayne,Key Colony Botanica,181 Crandon Blvd 205 Key Biscayne,"(181 Crandon Blvd APT 205, Key Biscayne, FL 33149, USA, (25.6977458, -80.16075560000002))","(25.6977458, -80.16075560000002, 0.0)",25.697746,-80.160756,0.0,1480000.0,925.00
7,St Tropez Condo 7330 Ocean Ter Ph-D Miami Beach,5/18/2022,8/17/2022,49.0,"$1,450,000.00",1410,"$1,028.37",Enrique Inurrieta,London Foster Realty,Enrique Inurrieta,London Foster Realty,St Tropez Condo 7330 Ocean Ter Ph-D Miami Beach,St Tropez Condo,7330 Ocean Ter Ph-D Miami Beach,"(7330 Ocean Terrace d, Miami Beach, FL 33141, USA, (25.8589938, -80.1201329))","(25.8589938, -80.1201329, 0.0)",25.858994,-80.120133,0.0,1450000.0,1028.37
8,Venice 13647 Deering Bay Dr 121 Coral Gables,5/6/2022,8/15/2022,55.0,"$1,350,000.00",2750,$490.91,Hazel Goldman,RE/MAX Advance Realty,Lindsey L

## Map URL Snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [134]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [135]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesSeptember2022
